In [3]:
import numpy as np

def rand_vec(size, sparsity):
    # Generate a random vector of size `size` with `sparsity` fraction of zeros
    # and the rest uniformly distributed between 0 and 1.
    # The vector is normalized to have unit length.
    
    vec = np.random.randint(0, 2, size)
    vec.astype(np.float32)
    return vec * np.random.uniform(0, 1, size)

rand_vec(100, 0.2)

array([0.        , 0.        , 0.        , 0.30174718, 0.17782506,
       0.4525299 , 0.        , 0.0053333 , 0.        , 0.        ,
       0.        , 0.34394999, 0.50436831, 0.93791526, 0.57450754,
       0.        , 0.94374842, 0.        , 0.        , 0.68777679,
       0.25621238, 0.        , 0.        , 0.60871735, 0.        ,
       0.1967035 , 0.        , 0.        , 0.10665403, 0.28372722,
       0.        , 0.52024502, 0.49798136, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.44433221, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.19416678,
       0.45283521, 0.        , 0.        , 0.        , 0.48755132,
       0.11223227, 0.        , 0.        , 0.71036638, 0.34535822,
       0.94852858, 0.        , 0.        , 0.        , 0.08308872,
       0.        , 0.67298497, 0.        , 0.        , 0.        ,
       0.        , 0.80452392, 0.        , 0.5430138 , 0.2214706 ,
       0.49912543, 0.64172781, 0.        , 0.63601993, 0.     

In [4]:
DIMS = 1000
SPARCITY = 0.1
vecs = [rand_vec(DIMS, SPARCITY) for i in range(10000)]
query = rand_vec(DIMS, SPARCITY)


exact_scores = [(i, np.dot(query, vec)) for i, vec in enumerate(vecs)]
exact_scores.sort(key=lambda x: x[1], reverse=True)
exact_scores[:10]

[(7076, 73.0911426864288),
 (2815, 72.19827548972923),
 (596, 71.96669054028024),
 (7008, 71.32413168126942),
 (7583, 71.05387918821906),
 (9310, 70.76266431972115),
 (208, 70.38511965546073),
 (6092, 70.21285363779795),
 (5574, 70.05242034964105),
 (1353, 70.00953595113604)]

In [10]:
class Weight:
    def __init__(self, idx, weight):
        self.idx = idx
        self.weight = weight
        
class SparseIndex:
    def __init__(self, vecs):
        self.vecs = np.array(vecs)
        columns = self.vecs.T
        self.dims = []
        for col in columns:
            dim = [Weight(i, weight) for i, weight in enumerate(col) if weight > 0]
            dim.sort(key=lambda x: x.weight)
            self.dims.append(dim)
            
    def _nearest_weights(self, lst, x, n):
        """Returns nearest n elements in list closest to index x"""
        ini = x - n
        end = x + n
        
        if ini < 0:
            end += -ini
            ini = 0
            
        if end > len(lst):
            end = len(lst)
            
        return lst[ini:end]
        
    def _select_ids(self, query, closest_n):
        """Find ids of vectors that are closer on a per-dimension basis to the query"""
        for dim in range(DIMS):
            col = self.dims[dim]
            lst = np.array([weight.idx for weight in col])
            idx = np.searchsorted(lst, query[dim])
            nearest_weights = self._nearest_weights(col, idx, closest_n)
            yield [weight.idx for weight in nearest_weights]
            
        
    
    def search(self, query, closest_n=3):
        deduped = set()
        for selected in self._select_ids(query, closest_n):
            for idx in selected:
                deduped.add(idx)
        print(f'collected {len(deduped)} candidates, choosing closest {closest_n}')
        
        vecs = []
        for idx in deduped:
            vecs.append((idx, self.vecs[idx]))
        
        scores = [(i, np.dot(query, vec)) for i, vec in vecs]
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores

    

In [11]:
sparse_index = SparseIndex(vecs)

approx_scores = sparse_index.search(query)
approx_scores[:10]

collected 4529 candidates, choosing closest 3


[(2815, 72.19827548972923),
 (596, 71.96669054028024),
 (7583, 71.05387918821906),
 (208, 70.38511965546073),
 (6092, 70.21285363779795),
 (2574, 69.48883406442897),
 (7736, 69.25668064699681),
 (1161, 69.07395953348099),
 (7049, 69.0492796233366),
 (3920, 68.88330861881681)]

In [12]:
exact_scores[:10]

[(7076, 73.0911426864288),
 (2815, 72.19827548972923),
 (596, 71.96669054028024),
 (7008, 71.32413168126942),
 (7583, 71.05387918821906),
 (9310, 70.76266431972115),
 (208, 70.38511965546073),
 (6092, 70.21285363779795),
 (5574, 70.05242034964105),
 (1353, 70.00953595113604)]

In [13]:
import plotly.express as px
import pandas as pd

exact = pd.DataFrame(exact_scores[:100], columns=['idx', 'score'])
exact['type'] = 'exact'
df = pd.DataFrame(exact)

for i in range(1, 21, 3):
    approx = pd.DataFrame(sparse_index.search(query, closest_n=i)[:100], columns=['idx', 'score'])
    approx['type'] = f'approx_closest_{i}'
    df = pd.concat([df, approx])

px.line(df, y='score', color='type', hover_data=['idx'], labels={'index': 'rank'})

collected 1823 candidates, choosing closest 1
collected 5529 candidates, choosing closest 4
collected 7571 candidates, choosing closest 7
collected 8681 candidates, choosing closest 10
collected 9289 candidates, choosing closest 13
collected 9626 candidates, choosing closest 16
collected 9781 candidates, choosing closest 19
